# Scrapping Digital Library ITB



## abstrak
salah satu nama proker di divisi himpunan saya: "Lex Luthor's Database" (tebak siapa yang ngusulkan nama itu). Secara sederhana, *#tidakmendetail*, proker ini bertugas untuk mencatat segala hal* ke sebuah database. Salah satunya, topik TA anak matematika. Tapi, jika anda membaca ini ketika notebook ini ditulis, website [Digital Library ITB](www.digilib.itb.ac.id) did not spark fun. Hal ini disebabkan karena bagian "Pencarian Lanjut" (advanced search)-nya error terus. Tapi, fear not! masih ada cara untuk mendapatkan informasi TA anak matematika dari website sialan ini.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# regular expression
import re

# object serialization
import pickle
import numpy as np
import pandas as pd 

# proses request dan edit HTML
import urllib.request as U
from bs4 import BeautifulSoup as bs

# pretty print
from pprint import pprint

# menampilkan proses iterasi
from tqdm import tqdm

pertama, kita perlu fungsi untuk mengambil data dari halaman digilib

In [0]:
def dictify(strnum):
  res = {}

  try:
    with U.urlopen('https://digilib.itb.ac.id/index.php/gdl/view/' + strnum) as f:
      raw = f.read().decode('utf8')
      soup = bs(raw, 'html.parser')

      tmp = []
      for p in soup.findAll('p')[1:]:
        # pakai try-except karna ada p tanpa class
        # cuma <p>
        try:
          if 'text-left' in p['class']:
            tmp2 = re.sub('(\n|\r|\t)', '', p.decode_contents())
            tmp2 = re.sub(' +', ' ', tmp2)
            tmp.append(tmp2.strip())
        except: pass
      res['judul'] = tmp[0]
      res['abstrak'] = tmp[1]

      tmp = soup.findAll('blockquote')[0]
      res['oleh'] = tmp.findAll('b')[0].decode_contents().strip()
                
      for tr in tmp.findAll('tr'):
        cols = [x.decode_contents() for x in tr.findAll('td')]
        key = cols[0]
        val = re.sub('(<. [^>]*>|<\/.>)', '', cols[2])
        val = re.sub('(\n|\r|\t)', '', val)
        val = re.sub(' +', ' ', val)

      if val!='':
        if key=='Kontributor / Dosen Pembimbing':  res['kontributor'] = val
        elif key=='Jenis Koleksi':                 res['jenis'] = val
        elif key=='Penerbit':                      res['penerbit'] = val
        elif key=='Fakultas':                      res['fakultas'] = val
        elif key=='Subjek':                        res['subjek'] = val
        elif key=='Kata Kunci':                    res['kunci'] = val
        elif key=='Tanggal Input':                 res['tanggal'] = val
  except: pass
  
  return res

lalu kita perlu fungsi untuk mengautomasi proses download. Karena program ini dijalankan di cloud, terdapat batasan waktu sekitar 2-3 jam sebelum proses ini dihentikan secara otomatis oleh sistem (dari sisi provider, tentunya mereka tidak ingin resource mereka terbuang karena satu *chunck* kode berjalan selama itu). Jadi, cara saya dengan mendownload 100 data lalu menyimpannya ke sebuah file pickle biar mudah digabungkan nantinya.

In [0]:
def batch_download(start, end, batch_size=100):
    for batch in range(start, end+1, batch_size):
        
        result = {}
        for num in tqdm(range(batch, batch+batch_size), desc=str(batch)):
            tmp = str(num)
            result[tmp] = dictify(tmp)

        with open('drive/My Drive/pickles/pickle_'+tmp, 'wb') as outfile:
            pickle.dump(result, outfile, -1)

In [0]:
#batch_download(33301, 33400)

selanjutnya kita tinggal menggabungkan data

In [0]:
result = {}
for n in tqdm(range(1, 420)):
  with open('drive/My Drive/pickles/pickle_{}00'.format(n), 'rb') as outfile:
    result.update(pickle.load(outfile))

data = pd.DataFrame.from_dict(result, orient='index')

100%|██████████| 419/419 [00:00<00:00, 542.12it/s]


In [0]:
data.head()

,judul,abstrak,oleh,kontributor,jenis,tanggal,penerbit,fakultas,kunci,subjek
1,A Design and An Implementation of Parallel Bat...,A,"Adang Suwandi, DR. IR.",A,Literatur Kelabu,2007-02-14 18:46:57,NaN,NaN,NaN,NaN
10,Hukum dan warfare,a,"Onno W. Purbo, PhD",#,Literatur Kelabu,2007-02-14 18:46:56,NaN,NaN,NaN,NaN
100,Murattal Quran: 110-An Nashr,MP3 File Audio.,Ghamiidi,digitized by Unit Salman.,Anggota,2004-09-15 16:24:44,NaN,NaN,"murattal quran, an nashr.",NaN
1000,STRUKTUR PASAR DAN PERFORMANSI INDUSTRI PERBAN...,S,Rina Indiastuti,P,S2 - Tesis,2017-09-27 14:50:29,FTI - Teknik dan Manajemen Industri,Fakultas Teknologi Industri (FTI),NaN,NaN
10000,FACTORS THAT INFLUENCES Y GREEN TEA CONSUMER P...,Persaingan di antara produsen minuman teh hija...,ARIA WIRA SAMUDERA (NIM 19005050),"Pembimbing : Drs. Herry Hudrasyah, M.A.",S1-Tugas Akhir,2009-07-03 16:07:42,Sekolah Bisnis dan Manajemen,Sekolah Bisnis dan Manajemen (SBM),"Preferensi, Fenomena, Konsumen, Faktor analisa",NaN


karena kita perlu mengubah data ini ke bentuk csv, kita perlu cari separator yang cocok untuk data ini.

In [0]:
'$' in data.to_string()

False

karena data ternyata punya karakter non UTF-8, kita terpaksa ngebuang karakter-karakter tersebut

In [0]:
with open('drive/My Drive/digi.csv', 'w', encoding='utf8', errors='replace') as f:
    data.to_csv(f, sep='$')